# Structured Outputs & Response Validation (Hands-On Tutorial)
Instructor: Zion Pibowei

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1jatO4DlPDWVcEVDA6hrzbTrphyShHwAs) [![Watch Video](https://img.shields.io/badge/Watch%20Video-4285F4?logo=googledrive&logoColor=white)](https://docs.google.com/document/d/1zwz6S-wDEQpRjufXYGeaUcXG_UB_M0M_aMz_FkD_IuM)


### What are Structured Outputs?
Structured outputs are LLM responses formatted in a specific, predictable schema (like JSON, XML, or defined data structures) rather than free-form text. This makes LLM outputs reliable, parseable and integration-ready.

#### Why Do We Need Them?
❌ Without Structure:
```
    LLM: "The user's name is John, he's 30 years old, and lives in New York."
```

- Hard to parse
- Inconsistent format
- Error-prone extraction


✅ With Structure:

```
  {
    "name": "John",
    "age": 30,
    "location": "New York"
  }
```

- Easy to parse
- Predictable format
- Type-safe

## 1. Basic Structured Outputs

### Prompt Engineering for JSON
The simplest approach is asking the LLM to output JSON:

In [ ]:
import os
from groq import Groq
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv(override=True)


client = Groq(api_key=os.getenv("GROQ_API_KEY"))

def get_json_output(prompt):
    response = client.chat.completions.create(
        model="llama-3.3-70b-versatile",
        messages=[
            {
                "role": "system",
                "content": "You are a helpful assistant that ONLY responds with valid JSON. Never include explanations, only JSON."
            },
            {
                "role": "user",
                "content": prompt
            }
        ],
        temperature=0.1
    )
    return response.choices[0].message.content

# Example usage
prompt = """
Extract the following information from this text and return as JSON:
Text: "John Adebayo is a 35-year-old software engineer living in Lagos. His email is john.adebayo@example.com"

Required fields:
- name (string)
- age (integer)
- occupation (string)
- city (string)
- email (string)
"""

result = get_json_output(prompt)
print(result)

### Parsing and Validating
Always validate LLM outputs

In [ ]:
import json
import re

def extract_and_parse_json(text):
    """Extract JSON from text that might contain markdown or extra content"""  

    try:
        # should run if valid JSON
        return json.loads(text)

    except json.JSONDecodeError as e:
        # Try to find JSON in code blocks
        json_match = re.search(r'```(?:json)?\s*(\{.*?\}|\[.*?\])\s*```', text, re.DOTALL)
        if json_match:
            json_str = json_match.group(1)
        else:
            # Try to find raw JSON
            json_match = re.search(r'(\{.*?\}|\[.*?\])', text, re.DOTALL)
            if json_match:
                json_str = json_match.group(1)
            else:
                raise ValueError("No JSON found in response")
        
        # print(json_str)
        return json.loads(json_str)
    
    except Exception as e:
        raise ValueError(f"Invalid JSON: {e}")

# Test it
response = """
Here's the extracted information:
```json
{
  "name": "John Adebayo",
  "age": 35,
  "occupation": "software engineer",
  "city": "Lagos",
  "email": "john.adebayo@example.com"
}
```
"""

data = extract_and_parse_json(response)
print(f"Name: {data['name']}, Age: {data['age']}")

## 2. Schema Validation with Pydantic

### Defining Data Models
Use Pydantic for robust validation

In [ ]:
# ! pip install pydantic[email]

In [ ]:
from pydantic import BaseModel, EmailStr, Field, validator
from typing import Optional, List
from datetime import datetime

class Person(BaseModel):
    """Structured person data model"""
    name: str = Field(..., min_length=1, max_length=100)
    age: int = Field(..., ge=0, le=150)
    email: EmailStr
    occupation: str
    city: str
    skills: Optional[List[str]] = []
    
    @validator('name')
    def name_must_be_capitalized(cls, v):
        if not v[0].isupper():
            raise ValueError('Name must start with capital letter')
        return v
    
    @validator('skills')
    def skills_must_not_be_empty(cls, v):
        if v is not None and len(v) == 0:
            return None
        return v

# Test validation
try:
    person = Person(
        name="John Adebayo",
        age=35,
        email="john.adebayo@example.com",
        occupation="Software Engineer",
        city="Lagos",
        skills=["Python", "JavaScript"]
    )
    print(f"Valid: {person}")
except Exception as e:
    print(f"Validation error: {e}")

# This will fail
try:
    invalid_person = Person(
        name="john",  # Not capitalized
        age=35,
        email="not-an-email",  # Invalid email
        occupation="Developer",
        city="NYC"
    )
except Exception as e:
    print(f"\nExpected validation error: {e}")

### LLM with Pydantic Validation
Combine LLM outputs with Pydantic

In [ ]:
import json
from typing import Type, TypeVar

T = TypeVar('T', bound=BaseModel)

def get_structured_output(prompt: str, model_class: Type[T], llm_client) -> T:
    """
    Get LLM output and validate against a Pydantic model
    """
    # Create schema-aware prompt
    schema = model_class.model_json_schema()
    
    full_prompt = f"""
{prompt}

You must respond with valid JSON matching this exact schema:
{json.dumps(schema, indent=2)}

Respond ONLY with the JSON object, nothing else.
"""
    
    response = llm_client.chat.completions.create(
        model="llama-3.3-70b-versatile",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that responds only with valid JSON."},
            {"role": "user", "content": full_prompt}
        ],
        temperature=0.1
    )
    
    # Parse response
    json_str = response.choices[0].message.content
    # print(f"LLM Repsonse: \n{json_str}")
    data = extract_and_parse_json(json_str)
    
    # Validate with Pydantic
    return model_class(**data)

# Usage 
prompt = """
Extract information about: "Alex Okafor is a 28-year-old data scientist in Abuja. 
His email is alex.okafor@company.com and he specializes in machine learning and Python."
"""

person = get_structured_output(prompt, Person, client)
print(f"Validated person: {person.name}, {person.age}, Skills: {person.skills}")

## 3. Complex Structured Outputs

### Nested Structures
Handle complex, nested data

In [ ]:
from pydantic import BaseModel, HttpUrl
from typing import List, Optional
from enum import Enum

class Priority(str, Enum):
    LOW = "low"
    MEDIUM = "medium"
    HIGH = "high"
    URGENT = "urgent"

class TaskStatus(str, Enum):
    TODO = "todo"
    IN_PROGRESS = "in_progress"
    COMPLETED = "completed"
    BLOCKED = "blocked"

class Subtask(BaseModel):
    title: str
    completed: bool = False
    
class Task(BaseModel):
    id: Optional[int] = None
    title: str = Field(..., min_length=1, max_length=200)
    description: Optional[str] = None
    priority: Priority
    status: TaskStatus = TaskStatus.TODO
    tags: List[str] = []
    subtasks: List[Subtask] = []
    due_date: Optional[str] = None  # Could use datetime with proper parsing
    assigned_to: Optional[str] = None

class ProjectPlan(BaseModel):
    project_name: str
    description: str
    tasks: List[Task]
    total_estimated_hours: Optional[float] = None

# Example usage
def extract_project_plan(text: str, llm_client) -> ProjectPlan:
    prompt = f"""
Analyze the following project description and create a structured project plan:

{text}

Extract:
- Project name and description
- List of tasks with priorities
- Any subtasks mentioned
- Tags for categorization
"""
    return get_structured_output(prompt, ProjectPlan, llm_client)

# Test
project_text = """
We're building a new e-commerce website. High priority tasks include:
1. Design the database schema (backend work)
2. Create user authentication system with login and signup pages (high priority, security)
3. Build product catalog (needs inventory integration and search functionality)
4. Implement shopping cart (medium priority)

This should take about 160 hours total.
"""

plan = extract_project_plan(project_text, client)
print(f"Project: {plan.project_name}")
for task in plan.tasks:
    print(f"- {task.title} [{task.priority}] - {len(task.subtasks)} subtasks")

### Multiple Output Formats
Support different structured formats

In [ ]:
from typing import Union, Literal

class JSONOutput(BaseModel):
    format: Literal["json"] = "json"
    data: dict

class TableOutput(BaseModel):
    format: Literal["table"] = "table"
    headers: List[str]
    rows: List[List[str]]

class MarkdownOutput(BaseModel):
    format: Literal["markdown"] = "markdown"
    content: str

OutputFormat = Union[JSONOutput, TableOutput, MarkdownOutput]

def get_flexible_output(prompt: str, output_format: str, llm_client):
    """Get output in user-specified format"""
    format_instructions = {
        "json": "Respond with a JSON object containing a 'data' field",
        "table": "Respond with JSON containing 'headers' (list) and 'rows' (list of lists)",
        "markdown": "Respond with JSON containing a 'content' field with markdown text"
    }
    
    full_prompt = f"""
{prompt}

{format_instructions[output_format]}

Format: {output_format}
"""
    
# Test with get_structured_output using appropriate model

## 4. Function Calling & Tool Use
### Native Function Calling
Modern LLMs support native function calling

## 4. Advanced Validation Techniques
### Retry Logic with Validation
Automatically retry when validation fails

In [ ]:
from tenacity import retry, stop_after_attempt, wait_exponential
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

@retry(
    stop=stop_after_attempt(3),
    wait=wait_exponential(multiplier=1, min=2, max=10),
    reraise=True
)
def get_validated_output(prompt: str, model_class: Type[T], llm_client, previous_error: str = None) -> T:
    """
    Get structured output with automatic retry on validation failure
    """
    # Add error feedback to prompt if this is a retry
    if previous_error:
        prompt = f"""
{prompt}

IMPORTANT: Your previous response had this validation error:
{previous_error}

Please correct the error and respond again with valid JSON.
"""
    
    try:
        return get_structured_output(prompt, model_class, llm_client)
    except Exception as e:
        logger.warning(f"Validation failed: {e}")
        # Store error for next attempt
        raise

# Usage with automatic retry
try:
    person = get_validated_output(
        "Extract: 'Alice, age 25, lives in Seattle'",
        Person,
        client
    )
    print(f"Success: {person}")
except Exception as e:
    print(f"Failed after retries: {e}")

### Partial Validation
Accept partially valid data

In [ ]:
from pydantic import BaseModel, ValidationError
from typing import Any, Dict

class PartialResult(BaseModel):
    valid_data: Dict[str, Any]
    invalid_fields: Dict[str, str]
    completeness_score: float

def validate_partial(data: dict, model_class: Type[BaseModel]) -> PartialResult:
    """
    Validate data partially, tracking which fields are valid/invalid
    """
    valid_data = {}
    invalid_fields = {}
    
    for field_name, field_info in model_class.model_fields.items():
        if field_name in data:
            try:
                # Try to validate individual field
                field_value = data[field_name]
                # Create temporary model with just this field
                temp_model = {field_name: field_value}
                model_class(**temp_model)
                valid_data[field_name] = field_value
            except ValidationError as e:
                invalid_fields[field_name] = str(e)
        else:
            invalid_fields[field_name] = "Missing field"
    
    total_fields = len(model_class.model_fields)
    valid_count = len(valid_data)
    completeness_score = valid_count / total_fields if total_fields > 0 else 0
    
    return PartialResult(
        valid_data=valid_data,
        invalid_fields=invalid_fields,
        completeness_score=completeness_score
    )

# Usage
incomplete_data = {
    "name": "Bob",
    "age": "not a number",  # Invalid
    "email": "bob@example.com"
    # Missing: occupation, city
}

result = validate_partial(incomplete_data, Person)
print(f"Completeness: {result.completeness_score:.0%}")
print(f"Valid fields: {list(result.valid_data.keys())}")
print(f"Invalid fields: {list(result.invalid_fields.keys())}")

### Custom Validators
Create domain-specific validators

In [ ]:
from pydantic import validator, field_validator
import re

class ProductData(BaseModel):
    name: str
    sku: str
    price: float
    category: str
    description: str
    
    @field_validator('sku')
    @classmethod
    def validate_sku_format(cls, v):
        """SKU must match pattern: XXX-YYYY-ZZ"""
        if not re.match(r'^[A-Z]{3}-\d{4}-\d{2}$', v):
            raise ValueError('SKU must match format XXX-YYYY-ZZ (e.g., ABC-1234-56)')
        return v
    
    @field_validator('price')
    @classmethod
    def validate_price(cls, v):
        """Price must be positive and reasonable"""
        if v <= 0:
            raise ValueError('Price must be positive')
        if v > 1_000_000:
            raise ValueError('Price seems unreasonably high')
        return round(v, 2)
    
    @field_validator('category')
    @classmethod
    def validate_category(cls, v):
        """Category must be from allowed list"""
        allowed = ['electronics', 'clothing', 'food', 'books', 'other']
        if v.lower() not in allowed:
            raise ValueError(f'Category must be one of: {", ".join(allowed)}')
        return v.lower()

# Test
try:
    product = ProductData(
        name="Laptop",
        sku="ELC-2024-01",
        price=999.99,
        category="electronics",
        description="High-performance laptop"
    )
    print(f"Valid product: {product.name}")
except ValidationError as e:
    print(f"Validation errors: {e}")

## 5. Production-Ready Implementation

### Complete Structured Output System - Example

In [ ]:
from typing import Type, Optional, Callable
from pydantic import BaseModel, ValidationError
import json
import logging
from datetime import datetime

class StructuredOutputConfig(BaseModel):
    max_retries: int = 3
    temperature: float = 0.1
    timeout: int = 30
    log_failures: bool = True

class OutputResult(BaseModel):
    success: bool
    data: Optional[Any] = None
    error: Optional[str] = None
    attempts: int = 1
    timestamp: datetime = datetime.now()

class StructuredOutputManager:
    """
    Production-ready manager for structured LLM outputs
    """
    def __init__(self, llm_client, config: StructuredOutputConfig = None):
        self.client = llm_client
        self.config = config or StructuredOutputConfig()
        self.logger = logging.getLogger(__name__)
    
    def extract(
        self,
        prompt: str,
        model_class: Type[BaseModel],
        system_prompt: Optional[str] = None,
        validator: Optional[Callable] = None
    ) -> OutputResult:
        """
        Extract structured data with validation and retry logic
        """
        attempts = 0
        last_error = None
        
        while attempts < self.config.max_retries:
            attempts += 1
            
            try:
                # Build prompt with schema
                schema = model_class.model_json_schema()
                full_prompt = self._build_prompt(prompt, schema, last_error)
                
                # Call LLM
                response = self._call_llm(full_prompt, system_prompt)
                
                # Parse and validate
                data = self._parse_response(response)
                validated_data = model_class(**data)
                
                # Custom validation if provided
                if validator:
                    validator(validated_data)
                
                # Success!
                return OutputResult(
                    success=True,
                    data=validated_data,
                    attempts=attempts
                )
                
            except Exception as e:
                last_error = str(e)
                self.logger.warning(f"Attempt {attempts} failed: {e}")
                
                if attempts >= self.config.max_retries:
                    if self.config.log_failures:
                        self.logger.error(f"All attempts failed: {last_error}")
                    
                    return OutputResult(
                        success=False,
                        error=last_error,
                        attempts=attempts
                    )
        
        return OutputResult(
            success=False,
            error="Max retries exceeded",
            attempts=attempts
        )
    
    def _build_prompt(self, prompt: str, schema: dict, error: Optional[str]) -> str:
        """Build prompt with schema and error feedback"""
        base_prompt = f"""
{prompt}

Respond with valid JSON matching this schema:
{json.dumps(schema, indent=2)}

CRITICAL: Respond ONLY with the JSON object. No explanations or markdown.
"""
        
        if error:
            base_prompt += f"""

Your previous response had this error: {error}
Please correct it and try again.
"""
        
        return base_prompt
    
    def _call_llm(self, prompt: str, system_prompt: Optional[str]) -> str:
        """Make LLM API call"""
        messages = []
        if system_prompt:
            messages.append({"role": "system", "content": system_prompt})
        else:
            messages.append({
                "role": "system",
                "content": "You are a helpful assistant that responds only with valid JSON."
            })
        
        messages.append({"role": "user", "content": prompt})
        
        response = self.client.chat.completions.create(
            model="llama-3.3-70b-versatile",
            messages=messages,
            temperature=self.config.temperature
        )
        
        return response.choices[0].message.content
    
    def _parse_response(self, response: str) -> dict:
        """Parse JSON from response"""
        return extract_and_parse_json(response)

# Usage
manager = StructuredOutputManager(client)

result = manager.extract(
    prompt="Extract: 'Sarah Johnson, 32, teacher in Austin, sarah.j@school.edu'",
    model_class=Person
)

if result.success:
    print(f"Success after {result.attempts} attempts")
    print(f"Data: {result.data}")
else:
    print(f"Failed after {result.attempts} attempts: {result.error}")

## 6. Real-World Examples

### E-commerce Product Extraction

In [ ]:
class ProductReview(BaseModel):
    rating: int = Field(..., ge=1, le=5)
    sentiment: Literal["positive", "negative", "neutral"]
    summary: str
    pros: List[str]
    cons: List[str]

class Product(BaseModel):
    name: str
    price: float
    description: str
    features: List[str]
    reviews: List[ProductReview]
    average_rating: float = Field(..., ge=1.0, le=5.0)

# Extract from unstructured data
text = """
Product: SuperPhone X1
Price: $899.99
Description: Latest flagship smartphone

Features:
- 6.5" OLED display
- 108MP camera
- 5000mAh battery

Reviews:
"Amazing phone! Camera is incredible. A bit pricey though." - 5 stars
"Good phone but battery life could be better" - 4 stars
"Love the display and performance!" - 5 stars
"""

result = manager.extract(
    prompt=f"Extract product information from:\n{text}",
    model_class=Product
)

print(result)

### Meeting Notes Extraction

In [ ]:
from datetime import date

class ActionItem(BaseModel):
    task: str
    assignee: str
    due_date: Optional[date] = None
    priority: Priority

class MeetingNotes(BaseModel):
    meeting_title: str
    date: date
    attendees: List[str]
    summary: str
    decisions: List[str]
    action_items: List[ActionItem]
    next_meeting: Optional[date] = None

# Extract from transcript
transcript = """
Team Standup - March 15, 2025
Attendees: Alice, Bob, Carol

Alice: We need to finalize the API documentation by Friday.
Bob: I'll handle that. Also, we should fix the login bug.
Carol: I can take the bug fix, high priority.
Alice: Great. Let's meet again next Tuesday.

Decisions:
- Move to microservices architecture
- Use PostgreSQL for the new module
"""

result = manager.extract(
    prompt=f"Extract meeting notes from:\n{transcript}",
    model_class=MeetingNotes
)

print(result)

## Key Takeaways - What have we learned?

✅ Best Practices

- Always use schemas - Define explicit data models
- Validate everything - Never trust LLM output blindly
- Use low temperature - 0.1-0.3 for consistent structured output
- Implement retries - LLMs can fail, build resilience
- Provide clear examples - Show the LLM what you want
- Log failures - Track and improve over time
- Use type hints - Make Python validate at development time
- Test edge cases - Missing fields, wrong types, etc.

❌ Common Pitfalls

- Assuming valid JSON - Always parse with try-catch
- Ignoring validation errors - They indicate real problems
- High temperature - Makes output inconsistent
- No retry logic - Single failures kill the app
- Complex nested structures - Start simple, add complexity gradually
- No partial validation - Sometimes incomplete data is useful
- Forgetting error feedback - Tell the LLM what went wrong